In [1]:
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/models"));
push!(LOAD_PATH, normpath(@__DIR__, "../../", "src/processing"));
ENV["COLUMNS"] = 200;

In [2]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [3]:
using BedsData
using ForecastData
using GeographicData

In [4]:
using PatientAllocation

In [5]:
states = ["CT", "MA", "NY", "PA"]

start_date = Date(2020, 5, 10)
end_date   = Date(2020, 6, 1)

pct_beds_available = 0.25
travel_threshold_hours = 4.0
hospitalized_days = 12;

In [6]:
counties = sort(get_counties(states))
county_names = get_county_names(counties);

In [7]:
N = length(counties);
T = (end_date - start_date).value + 1;

In [8]:
forecast_admitted = forecast(
    counties, start_date, end_date,
    level=:county,
    source=:columbia,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
);

In [9]:
forecast_initial = forecast(
    counties, start_date-Dates.Day(1), start_date-Dates.Day(1),
    level=:county,
    source=:columbia,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
    hospitalized_days=hospitalized_days,
)[:];

In [10]:
forecast_discharged = forecast(
    counties, start_date-Dates.Day(hospitalized_days), start_date-Dates.Day(1),
    level=:county,
    source=:columbia,
    forecast_type=:admitted,
    patient_type=:regular,
    bound_type=:mean,
)
forecast_discharged = hcat(forecast_discharged, zeros(Float32, N, T - hospitalized_days));

In [11]:
adj = adjacencies(counties, level=:county, source=:latlong);

In [12]:
beds = n_beds(counties, level=:county, bed_type=:all, pct_beds_available=pct_beds_available);

In [13]:
DataFrame(
    county=county_names,
    beds=beds,
    initial=forecast_initial
)

,county,beds,initial
,String,Float32,Float32
1,Fairfield CT,403.0,356.0
2,Hartford CT,722.0,1271.0
3,Litchfield CT,39.25,183.0
4,Middlesex CT,75.0,38.0
5,New Haven CT,421.25,765.0
6,New London CT,104.75,65.0
7,Tolland CT,38.0,18.0
8,Windham CT,37.0,31.0
9,Barnstable MA,83.5,233.0


In [14]:
model = patient_allocation(
    beds,
    forecast_initial,
    forecast_admitted,
    forecast_discharged,
    adj,
    hospitalized_days=hospitalized_days,
    send_new_only=true,
    sendrecieve_switch_time=0,
    min_send_amt=0,
    smoothness_penalty=0,
    setup_cost=0,
    sent_penalty=0,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 540755 rows, 527896 columns and 23201551 nonzeros
Model fingerprint: 0x82ebc9ba
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-01, 6e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 532588 rows and 389916 columns (presolve time = 6s) ...
Presolve removed 534913 rows and 404536 columns
Presolve time: 11.76s
Presolved: 5842 rows, 123360 columns, 4908749 nonzeros

Ordering time: 0.09s

Barrier statistics:
 AA' NZ     : 8.143e+06
 Factor NZ  : 1.480e+07 (roughly 170 MBytes of memory)
 Factor Ops : 4.984e+10 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Prim

In [15]:
overflow_per_day = (i,t) -> sum(max.(0,
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t-1])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
    - beds[i])
)
overflow = i -> sum(overflow_per_day(i,t) for t=1:size(sent,3));

In [16]:
println("Total overflow: ", sum(overflow.(1:length(counties))))

Total overflow: 357683.25


In [17]:
summary = DataFrame(
    county=county_names,
    total_sent=sum(sent, dims=[2,3])[:],
    total_received=sum(sent, dims=[1,3])[:],
    overflow=overflow.(1:N),
)

,county,total_sent,total_received,overflow
,String,Float64,Float64,Float64
1,Fairfield CT,50.0,412.0,166.75
2,Hartford CT,566.0,0.0,2658.0
3,Litchfield CT,71.75,6.0,834.75
4,Middlesex CT,17.0,69.0,234.0
5,New Haven CT,462.75,0.0,1417.75
6,New London CT,3.0,169.5,91.75
7,Tolland CT,3.0,49.0,172.0
8,Windham CT,2.0,45.0,103.0
9,Barnstable MA,163.75,0.0,943.5


In [18]:
sent_matrix = DataFrame(sum(sent, dims=3)[:,:,1])
rename!(sent_matrix, Symbol.(county_names))
insertcols!(sent_matrix, 1, :county => county_names)

,county,Fairfield CT,Hartford CT,Litchfield CT,Middlesex CT,New Haven CT,New London CT,Tolland CT,Windham CT,Barnstable MA,Berkshire MA,Bristol MA,Dukes MA,Essex MA,Franklin MA,Hampden MA
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Fairfield CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
2,Hartford CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0
3,Litchfield CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Middlesex CT,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,New Haven CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,New London CT,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Tolland CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Windham CT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,Barnstable MA,8.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0


In [19]:
sent_vis_matrix = sum(sent, dims=3)[:,:,1] + diagm(sum(max.(0, forecast_admitted), dims=2)[:] - sum(sent, dims=[2,3])[:])
sent_vis_matrix = DataFrame(sent_vis_matrix)
rename!(sent_vis_matrix, Symbol.(county_names));

In [20]:
total_patients = (i,t) -> (
    forecast_initial[i] - sum(forecast_discharged[i,1:min(t,hospitalized_days)])
    + sum(forecast_admitted[i,max(1,t-hospitalized_days):t])
    - sum(sent[i,:,1:t])
    + sum(sent[:,i,max(1,t-hospitalized_days):t])
);

In [21]:
outcomes = DataFrame()
for (i,s) in enumerate(county_names)
    single_state_outcome = DataFrame(
        county=fill(s, T),
        day=start_date .+ Dates.Day.(0:T-1),
        sent=sum(sent[i,:,:], dims=1)[:],
        received=sum(sent[:,i,:], dims=1)[:],
        new_patients=forecast_admitted[i,:],
        total_patients=[total_patients(i,t) for t in 1:T],
        capacity=fill(beds[i], T),
        overflow=[overflow_per_day(i,t) for t in 1:T],
        sent_to=[sum(sent[i,:,t])>0 ? collect(zip(county_names[sent[i,:,t] .> 0], sent[i,sent[i,:,t].>0,t])) : "[]" for t in 1:T],
        received_from=[sum(sent[:,i,t])>0 ? collect(zip(county_names[sent[:,i,t] .> 0], sent[sent[:,i,t].>0,i,t])) : "[]" for t in 1:T],
    )
    outcomes = vcat(outcomes, single_state_outcome)
end
# CSV.write("patient_allocation_results.csv", outcomes)
println("First day:")
filter(row -> row.day == start_date, outcomes)

First day:


,county,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any
1,Fairfield CT,2020-05-10,10.0,105.25,10.0,391.25,403.0,0.0,"[(""Philadelphia PA"", 10.0)]"
2,Hartford CT,2020-05-10,91.0,0.0,91.0,1142.0,722.0,511.0,"[(""Nassau NY"", 91.0)]"
3,Litchfield CT,2020-05-10,11.0,0.0,11.0,164.0,39.25,135.75,"[(""Centre PA"", 11.0)]"
4,Middlesex CT,2020-05-10,3.0,38.0,3.0,72.0,75.0,0.0,"[(""Nassau NY"", 3.0)]"
5,New Haven CT,2020-05-10,51.0,0.0,51.0,684.0,421.25,313.75,"[(""Ontario NY"", 51.0)]"
6,New London CT,2020-05-10,2.0,0.0,2.0,58.0,104.75,0.0,"[(""Middlesex CT"", 2.0)]"
7,Tolland CT,2020-05-10,1.0,0.0,1.0,16.0,38.0,0.0,"[(""Montgomery NY"", 1.0)]"
8,Windham CT,2020-05-10,1.0,4.0,1.0,32.0,37.0,0.0,"[(""Dukes MA"", 1.0)]"
9,Barnstable MA,2020-05-10,21.0,0.0,21.0,216.0,83.5,153.5,"[(""Schenectady NY"", 21.0)]"


In [22]:
s = "Fairfield CT"
filter(row -> row.county == s, outcomes)

,county,day,sent,received,new_patients,total_patients,capacity,overflow,sent_to,received_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Any,Any
1,Fairfield CT,2020-05-10,10.0,105.25,10.0,391.25,403.0,0.0,"[(""Philadelphia PA"", 10.0)]","[(""Kings NY"", 105.25)]"
2,Fairfield CT,2020-05-11,12.0,54.75,12.0,391.0,403.0,0.0,"[(""Oswego NY"", 1.0), (""Cumberland PA"", 11.0)]","[(""Middlesex CT"", 4.0), (""Queens NY"", 50.75)]"
3,Fairfield CT,2020-05-12,0.0,45.0,8.0,403.0,403.0,0.0,[],"[(""Hampden MA"", 45.0)]"
4,Fairfield CT,2020-05-13,0.0,26.0,9.0,411.0,403.0,8.0,[],"[(""Norfolk MA"", 26.0)]"
5,Fairfield CT,2020-05-14,0.0,0.0,11.0,403.0,403.0,0.0,[],[]
6,Fairfield CT,2020-05-15,0.0,0.0,11.0,403.0,403.0,0.0,[],[]
7,Fairfield CT,2020-05-16,0.0,0.0,10.0,403.0,403.0,0.0,[],[]
8,Fairfield CT,2020-05-17,0.0,0.0,11.0,407.0,403.0,4.0,[],[]
9,Fairfield CT,2020-05-18,0.0,8.75,10.0,418.75,403.0,15.75,[],"[(""Barnstable MA"", 8.75)]"


In [23]:
println("Sent to:")
Dict(county_names[i] => county_names[row] for (i,row) in enumerate(eachrow(sum(sent, dims=3)[:,:,1] .> 0)))

Sent to:


Dict{String,Array{String,1}} with 151 entries:
  "Yates NY"          => String[]
  "Richmond NY"       => ["Hampden MA", "Middlesex MA", "Plymouth MA", "Cambria PA", "Lehigh PA"]
  "Centre PA"         => String[]
  "Beaver PA"         => ["Crawford PA", "Lawrence PA"]
  "Rensselaer NY"     => ["Columbia NY", "Bradford PA"]
  "Ontario NY"        => String[]
  "Bucks PA"          => ["Hampden MA", "Middlesex MA", "Worcester MA", "Cattaraugus NY", "Genesee NY", "Monroe NY", "Queens NY", "Sullivan NY", "Wyoming NY", "Dauphin PA", "Fayette PA", "Indiana PA"…
  "Northumberland PA" => String[]
  "Lycoming PA"       => String[]
  "Indiana PA"        => String[]
  "Herkimer NY"       => String[]
  "Monroe PA"         => ["Chautauqua NY", "Lehigh PA", "Westmoreland PA"]
  "Washington NY"     => String[]
  "Oneida NY"         => ["Clarion PA"]
  "St. Lawrence NY"   => String[]
  "Warren NY"         => String[]
  "Westmoreland PA"   => ["Armstrong PA", "Somerset PA"]
  "Franklin NY"       => Strin